In [0]:
from pyspark.sql.functions import when
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("CDC").getOrCreate()

In [0]:
fldf = spark.read.csv("/FileStore/tables/LOAD00000001.csv")
fldf = fldf.selectExpr("_c0 as id","_c1 as FullName","_c2 as City")
fldf.write.mode("overwrite").csv("/FileStore/tables/finalOutput/finalFile.csv")

In [0]:
udf = spark.read.csv("/FileStore/tables/20220927_071840337.csv")
udf = udf.selectExpr("_c0 as action","_c1 as id","_c2 as FullName","_c3 as City")
ffdf = spark.read.csv("/FileStore/tables/finalOutput/finalFile.csv")
ffdf = ffdf.selectExpr("_c0 as id", "_c1 as FullName","_c2 as City" )
display(ffdf)

id,FullName,City
0,Herman Zimmerman,Oklahoma City
1,Lisa Ray,Columbus
2,Terrell Reeves,Jacksonville
3,Steve Goodwin,Charlotte
4,Leah Tran,Detroit
5,Wilbert Holmes,Washington
6,Mindy George,Los Angeles
7,Rosa Huff,Phoenix
8,Clayton Jennings,Denver
9,Darla Hayes,Charlotte


In [0]:
for row in udf.collect():
  
    if row["action"] == 'U':
        ffdf = ffdf.withColumn("FullName", when(ffdf["id"] == row["id"], row["FullName"]).otherwise(ffdf["FullName"]))
        ffdf = ffdf.withColumn("City", when(ffdf["id"] == row["id"],row["City"]).otherwise(ffdf["City"]))
        
    if row["action"] == 'I':
        insertedRow = [list(row)[1:]]
        columns = ['id', 'FullName', 'City']
        newdf = spark.createDataFrame(insertedRow, columns)
        ffdf = ffdf.union(newdf)
        
    if row["action"] == 'D':
        ffdf = ffdf.filter(ffdf.id != row["id"])

In [0]:
display(ffdf)

id,FullName,City
0,Herman Zimmerman,Oklahoma City
1,Lisa Ray,Columbus
2,Terrell Reeves,Jacksonville
3,Steve Goodwin,Charlotte
4,Leah Tran,Detroit
5,Wilbert Holmes,Washington
6,Mindy George,Los Angeles
7,ABC XYZ,Phoenix
8,ABC XYZ,Denver
9,Darla Hayes,Charlotte


In [0]:
ffdf.write.mode("overwrite").csv("/FileStore/tables/finalOutput/finalFile.csv")